# Basic Tools for Alhazen

> Simple tools to demonstrate utility and 'agentic' functionality. 

## Tools:
1. Develop queries across external data sources
2. Execute a query against EPMC, creating a collection and saving expressions, items, and fragments to the database
3. Add all available full text to a collection
4. Filter a collection by an LLM-based analysis by tagging fragments with Notes
5. Extract information from a collection using an LLM-based analysis to create Notes 
6. Report on the state of the database in terms of numbers of collections, expressions, items, and fragments
7. Prepare a report over a core research question by collecting a number of notes and synthesizing them into a report

## Workflow
![Workflow](https://lucid.app/publicSegments/view/fea24e0a-61c7-4807-8ea2-e4859753c31b/image.png)

In [ ]:
#| default_exp tools.basic_tools

In [1]:
#| export

import os

from typing import Optional, Type

from langchain.tools import StructuredTool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentType, initialize_agent
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.utilities import SerpAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field

from alhazen.utils.airtableUtils import AirtableUtils
from alhazen.utils.searchEngineUtils import ESearchQuery, EuroPMCQuery
from alhazen.utils.queryTranslator import QueryTranslator, QueryType
from alhazen.schema_sqla import ScientificKnowledgeCollection, \
    ScientificKnowledgeExpression, ScientificKnowledgeCollectionHasMembers, \
    ScientificKnowledgeItem, ScientificKnowledgeExpressionHasRepresentation, \
    ScientificKnowledgeFragment, ScientificKnowledgeItemHasPart, \
    InformationResource, Note, NoteIsAbout
from alhazen.utils.local_literature_db import QuerySpec, LocalLiteratureDb
from alhazen.utils.jats_text_extractor import NxmlDoc

import pandas as pd

from sqlalchemy import create_engine, exists, func
from sqlalchemy.orm import sessionmaker



In [11]:
#| export

class EMPCSearchToolSchema(BaseModel):
    query: str = Field(description="should be a search query")
    name: str = Field(description="should be the name of the collection we will add papers to")

class EMPCSearchTool(BaseTool):
    name = "epmc_search"
    description = "useful for when you need to search for biomedical scientific papers"
    args_schema: Type[EMPCSearchToolSchema] = EMPCSearchToolSchema

    def _run(
        self,
        name: str,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        
        """Use the tool."""
        if os.environ.get('LLMS_TEMP_DIR') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LLMS_TEMP_DIR']
        if os.environ.get('DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['DB_NAME']
        db = LocalLiteratureDb(loc, db_name)
        if db.session is None:
            session_class = sessionmaker(bind=db.engine)
            db.session = session_class()

        r = db.session.query(func.max(ScientificKnowledgeCollection.id)).first()
        max_id = r[0]
        if max_id is None:
            c_id = 0
        else:
            c_id = int(max_id) + 1

        cdf = pd.DataFrame([{'ID': c_id, 'NAME': name, 'QUERY': query}])        
        qs = QuerySpec(db_name, 'ID', 'QUERY', 'NAME', {}, ['TITLE','ABSTRACT'])
        qt = QueryTranslator(cdf.sort_values('ID'), 'ID', 'QUERY', 'NAME')
        
        db.add_corpus_from_epmc(qt, None, sections=qs.sections)

        r2 = db.session.query(func.count(ScientificKnowledgeExpression.id)) \
                .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id) \
                .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id) \
                .filter(ScientificKnowledgeCollection.id == c_id).first()
        n_papers_added = r2[0]

        r2 = db.session.query(func.count(ScientificKnowledgeExpression.id)) \
                .filter(ScientificKnowledgeCollection.id == ScientificKnowledgeCollectionHasMembers.ScientificKnowledgeCollection_id) \
                .filter(ScientificKnowledgeCollectionHasMembers.has_members_id == ScientificKnowledgeExpression.id) \
                .filter(ScientificKnowledgeCollection.id == c_id).first()
        n_papers_added = r2[0]

        return {'action': 'Final Answer', 
                'action_input': 'Successfully constructed a collection called `{}` containing {} papers by querying EMPC'.format(name, n_papers_added)}

    async def _arun(
        self,
        name: str,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("epmc_search does not support async")

In [12]:
os.environ['LLMS_TEMP_DIR'] = '/tmp/alhazen/'
os.environ['DB_NAME'] = 'temp'

t = EMPCSearchTool()
t._run(name='More Stuff4', query='Cryoelectron Tomography | Cryo Electron Tomography | Cryo-Electron Tomography | Cryo-ET | CryoE')


100%|██████████| 1/1 [00:00<00:00, 1440.85it/s]


https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize=1000&synonym=TRUE&resultType=core&query=((TITLE:"Cryoelectron Tomography" OR ABSTRACT:"Cryoelectron Tomography") OR (TITLE:"Cryo Electron Tomography" OR ABSTRACT:"Cryo Electron Tomography") OR (TITLE:"Cryo-Electron Tomography" OR ABSTRACT:"Cryo-Electron Tomography") OR (TITLE:"Cryo-ET" OR ABSTRACT:"Cryo-ET") OR (TITLE:"CryoE" OR ABSTRACT:"CryoE")), 1847 European PMC PAPERS FOUND


100%|██████████| 2/2 [00:31<00:00, 15.97s/it]


 Returning 1847


100%|██████████| 1847/1847 [00:06<00:00, 305.37it/s]


{'action': 'Final Answer',
 'action_input': 'Successfully constructed a collection called More Stuff4 containing 1847 papers from EPMC'}